In [261]:
import pandas as pd

In [262]:
file = pd.read_excel("ChallengeData-Sandesh.xlsx",header=2)#Read the file

In [263]:
#Target Variables
target = ['Revenue - Tortoise','Revenue - Rabbit', 'Average Revenue per existing customer Excl Line Rental - Rabbit', 'Average Revenue per existing customer Excl Line Rental - Tortoise']

In [264]:
df = pd.DataFrame()

In [265]:
#Select the target rows
for i in target:
    if df.empty:
        df = file[file['Generic Variable']== i]
    else:
        df = df.append(file[file['Generic Variable']== i])

In [266]:
#Drop the useless columns
df = df.drop(['Generic Group','Generic Product ','Generic Group variable','Generic Brand','Generic Product Category','Generic Variable','Generic Sub-Variable','Units'],axis=1)

In [267]:
#Drop the nan columns
df.dropna(axis=1,inplace=True)

In [268]:
df1 = pd.DataFrame()

In [269]:
#Data Processing，turn Target Variable to Characteristics
for i in range(len(df)):
    Dict = {'Generic LookupKey':[],'time':[],'target':[]}
    for j in df:
        if not (j in Dict):
            Dict['time'].append(j)
            Dict['target'].append(df.iloc[i][j])
            Dict['Generic LookupKey'].append(df.iloc[i]['Generic LookupKey'])
    df2 = pd.DataFrame(Dict)
    df2['t-1'] = df2['target'].shift(1)
    df2['t-2'] = df2['target'].shift(2)
      # I use the values of two months ahead to predict the value of the third month
    df2.dropna(axis=0,inplace=True)
    if df1.empty:
        df1 = df2
    else:
        df1 = pd.concat([df1,df2])
    del df2

In [271]:
df1 = pd.get_dummies(df1,columns=['Generic LookupKey'])
#One-hot encoding for Target Variables

In [273]:
year = []
month =[]
for x in df1['time']:
    year.append(x.year)
    month.append(x.month) 
df1['year']=year
df1['month']=month

In [274]:
df1.drop(['time'],axis=1,inplace=True)

In [276]:
df1 = pd.get_dummies(df1,columns=['month'])
#one-hot encoding for month

In [277]:
x = df1.drop(['target'],axis=1)

In [278]:
y = df1['target']

In [279]:
from sklearn.model_selection import train_test_split

In [280]:
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import VotingRegressor

In [281]:
import numpy as np

def mape(estimator,x,y_true):
    y_pred = estimator.predict(x)
    return np.mean(np.abs((y_pred - y_true) / y_true))*0.7+0.3*np.max(np.abs((y_pred-y_true)/y_true))

In [282]:
#parameters are set by GridSearch Cross-Validation
xgbr = XGBRegressor(objective = 'reg:squarederror',eval_metric='mae',max_depth=3,n_estimators=200,booster='dart')
rfr = RandomForestRegressor(n_estimators=600,max_depth=16,criterion='mse')
model = VotingRegressor([('XGBoost',xgbr),('RandomForest',rfr)],weights=[0.3,0.7])

In [283]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.25)

In [284]:
model.fit(x_train,y_train)

C:\Users\aSUS\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


VotingRegressor(estimators=[('XGBoost',
                             XGBRegressor(base_score=0.5, booster='dart',
                                          colsample_bylevel=1,
                                          colsample_bynode=1,
                                          colsample_bytree=1, eval_metric='mae',
                                          gamma=0, importance_type='gain',
                                          learning_rate=0.1, max_delta_step=0,
                                          max_depth=3, min_child_weight=1,
                                          missing=None, n_estimators=200,
                                          n_jobs=1, nthread=None,
                                          objective='reg:squarederror',
                                          random_state=0...
                            ('RandomForest',
                             RandomForestRegressor(bootstrap=True,
                                                   criterion='mse',

In [285]:
#Local cross-validation
mape(model,x_test,y_test)

1.5519257181856094

In [286]:
#Use the whold data to train
model.fit(x,y)

C:\Users\aSUS\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\aSUS\Anaconda3\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


VotingRegressor(estimators=[('XGBoost',
                             XGBRegressor(base_score=0.5, booster='dart',
                                          colsample_bylevel=1,
                                          colsample_bynode=1,
                                          colsample_bytree=1, eval_metric='mae',
                                          gamma=0, importance_type='gain',
                                          learning_rate=0.1, max_delta_step=0,
                                          max_depth=3, min_child_weight=1,
                                          missing=None, n_estimators=200,
                                          n_jobs=1, nthread=None,
                                          objective='reg:squarederror',
                                          random_state=0...
                            ('RandomForest',
                             RandomForestRegressor(bootstrap=True,
                                                   criterion='mse',

In [287]:
months = 12 # No. of months to predict

In [288]:
last_month = df.columns[-1]

In [289]:
type(last_month)

datetime.datetime

In [302]:
ans = {'Generic LookupKey':[],'Time Period':[],'Value':[]}

In [303]:
ans

{'Generic LookupKey': [], 'Time Period': [], 'Value': []}

In [304]:
df.columns

Index(['Generic LookupKey', 2016-04-01 00:00:00, 2016-05-01 00:00:00,
       2016-06-01 00:00:00, 2016-07-01 00:00:00, 2016-08-01 00:00:00,
       2016-09-01 00:00:00, 2016-10-01 00:00:00, 2016-11-01 00:00:00,
       2016-12-01 00:00:00, 2017-01-01 00:00:00, 2017-02-01 00:00:00,
       2017-03-01 00:00:00, 2017-04-01 00:00:00, 2017-05-01 00:00:00,
       2017-06-01 00:00:00, 2017-07-01 00:00:00, 2017-08-01 00:00:00,
       2017-09-01 00:00:00, 2017-10-01 00:00:00, 2017-11-01 00:00:00,
       2017-12-01 00:00:00, 2018-01-01 00:00:00, 2018-02-01 00:00:00,
       2018-03-01 00:00:00, 2018-04-01 00:00:00, 2018-05-01 00:00:00,
       2018-06-01 00:00:00, 2018-07-01 00:00:00, 2018-08-01 00:00:00,
       2018-09-01 00:00:00, 2018-10-01 00:00:00, 2018-11-01 00:00:00,
       2018-12-01 00:00:00, 2019-01-01 00:00:00, 2019-02-01 00:00:00,
       2019-03-01 00:00:00],
      dtype='object')

In [305]:
for i in range(len(df)):
    f0 = df.iloc[i][-2]
    f1 = df.iloc[i][-1]
    res= []
    for j in range(months):
        d = {'Generic LookupKey':[df.iloc[i]['Generic LookupKey']],'t-2':f0,'t-1':f1}
        df3 = pd.DataFrame(d) 
        df3 = pd.get_dummies(df3,columns=['Generic LookupKey'])
        for k in x.columns:
            if not (k in df3.columns):
                df3[k] = 0
        df3 = df3[x.columns]
   #     print(df3['t-1'])
        y = model.predict(df3)
        print(y)
        res.append(y)
        f0 = f1
        f1 = y
    time = last_month
    for z in res:
        ans['Generic LookupKey'].append(df.iloc[i]['Generic LookupKey'])
        year = time.year
        month = time.month
        if month==12:
            month=1
            year+=1
        else:
            month+=1
        time = datetime(year,month,1)
        ans['Time Period'] .append(time)
        ans['Value'].append(z[0])

[-0.88069506]
[-0.89825287]
[-0.90762476]
[-0.89505719]
[-0.90079416]
[-0.90388379]
[-0.89561413]
[-0.90136016]
[-0.90388379]
[-0.89561413]
[-0.90136016]
[-0.90388379]
[0.89746155]
[0.85966976]
[0.74299006]
[0.74783338]
[0.74231851]
[0.73560859]
[0.73411515]
[0.73550477]
[0.73398163]
[0.73550477]
[0.73398163]
[0.73550477]
[-0.108741]
[-0.11691274]
[-0.12054626]
[-0.26812127]
[-0.26759065]
[-0.23900698]
[-0.19485873]
[-0.1843671]
[-0.20154028]
[-0.19959413]
[-0.1906392]
[-0.18852889]
[-0.70828672]
[-0.74336413]
[-0.72007678]
[-0.71230784]
[-0.71680103]
[-0.71547002]
[-0.71682844]
[-0.71547002]
[-0.71682844]
[-0.71547002]
[-0.71682844]
[-0.71547002]


In [306]:
result = pd.DataFrame(ans)

In [307]:
result

Generic LookupKey Time Period     Value
0   Segment 1 - Sandesh Brand 2Sandesh Brand 2Broa...  2019-04-01 -0.880695
1   Segment 1 - Sandesh Brand 2Sandesh Brand 2Broa...  2019-05-01 -0.898253
2   Segment 1 - Sandesh Brand 2Sandesh Brand 2Broa...  2019-06-01 -0.907625
3   Segment 1 - Sandesh Brand 2Sandesh Brand 2Broa...  2019-07-01 -0.895057
4   Segment 1 - Sandesh Brand 2Sandesh Brand 2Broa...  2019-08-01 -0.900794
5   Segment 1 - Sandesh Brand 2Sandesh Brand 2Broa...  2019-09-01 -0.903884
6   Segment 1 - Sandesh Brand 2Sandesh Brand 2Broa...  2019-10-01 -0.895614
7   Segment 1 - Sandesh Brand 2Sandesh Brand 2Broa...  2019-11-01 -0.901360
8   Segment 1 - Sandesh Brand 2Sandesh Brand 2Broa...  2019-12-01 -0.903884
9   Segment 1 - Sandesh Brand 2Sandesh Brand 2Broa...  2020-01-01 -0.895614
10  Segment 1 - Sandesh Brand 2Sandesh Brand 2Broa...  2020-02-01 -0.901360
11  Segment 1 - Sandesh Brand 2Sandesh Brand 2Broa...  2020-03-01 -0.903884
12  Segment 1 - Sandesh Brand 2Sandesh Brand 2Broa...  2019-04-01  0.897462
13  Segment 1 - Sandesh Brand 2Sandesh Brand 2Broa...  2019-05-01  0.859670
14  Segment 1 - Sandesh Brand 2Sandesh Brand 2Broa...  2019-06-01  0.742990
15  Segment 1 - Sandesh Brand 2Sandesh Brand 2Broa...  2019-07-01  0.747833
16  Segment 1 - Sandesh Brand 2Sandesh Brand 2Broa...  2019-08-01  0.742319
17  Segment 1 - Sandesh Brand 2Sandesh Brand 2Broa...  2019-09-01  0.735609
18  Segment 1 - Sandesh Brand 2Sandesh Brand 2Broa...  2019-10-01  0.734115
19  Segment 1 - Sandesh Brand 2Sandesh Brand 2Broa...  2019-11-01  0.735505
20  Segment 1 - Sandesh Brand 2Sandesh Brand 2Broa...  2019-12-01  0.733982
21  Segment 1 - Sandesh Brand 2Sandesh Brand 2Broa...  2020-01-01  0.735505
22  Segment 1 - Sandesh Brand 2Sandesh Brand 2Broa...  2020-02-01  0.733982
23  Segment 1 - Sandesh Brand 2Sandesh Brand 2Broa...  2020-03-01  0.735505
24  Segment 1 - Sandesh Brand 2Sandesh Brand 2Broa...  2019-04-01 -0.108741
25  Segment 1 - Sandesh Brand 2Sandesh Brand 2Broa...  2019-05-01 -0.116913
26  Segment 1 - Sandesh Brand 2Sandesh Brand 2Broa...  2019-06-01 -0.120546
27  Segment 1 - Sandesh Brand 2Sandesh Brand 2Broa...  2019-07-01 -0.268121
28  Segment 1 - Sandesh Brand 2Sandesh Brand 2Broa...  2019-08-01 -0.267591
29  Segment 1 - Sandesh Brand 2Sandesh Brand 2Broa...  2019-09-01 -0.239007
30  Segment 1 - Sandesh Brand 2Sandesh Brand 2Broa...  2019-10-01 -0.194859
31  Segment 1 - Sandesh Brand 2Sandesh Brand 2Broa...  2019-11-01 -0.184367
32  Segment 1 - Sandesh Brand 2Sandesh Brand 2Broa...  2019-12-01 -0.201540
33  Segment 1 - Sandesh Brand 2Sandesh Brand 2Broa...  2020-01-01 -0.199594
34  Segment 1 - Sandesh Brand 2Sandesh Brand 2Broa...  2020-02-01 -0.190639
35  Segment 1 - Sandesh Brand 2Sandesh Brand 2Broa...  2020-03-01 -0.188529
36  Segment 1 - Sandesh Brand 2Sandesh Brand 2Broa...  2019-04-01 -0.708287
37  Segment 1 - Sandesh Brand 2Sandesh Brand 2Broa...  2019-05-01 -0.743364
38  Segment 1 - Sandesh Brand 2Sandesh Brand 2Broa...  2019-06-01 -0.720077
39  Segment 1 - Sandesh Brand 2Sandesh Brand 2Broa...  2019-07-01 -0.712308
40  Segment 1 - Sandesh Brand 2Sandesh Brand 2Broa...  2019-08-01 -0.716801
41  Segment 1 - Sandesh Brand 2Sandesh Brand 2Broa...  2019-09-01 -0.715470
42  Segment 1 - Sandesh Brand 2Sandesh Brand 2Broa...  2019-10-01 -0.716828
43  Segment 1 - Sandesh Brand 2Sandesh Brand 2Broa...  2019-11-01 -0.715470
44  Segment 1 - Sandesh Brand 2Sandesh Brand 2Broa...  2019-12-01 -0.716828
45  Segment 1 - Sandesh Brand 2Sandesh Brand 2Broa...  2020-01-01 -0.715470
46  Segment 1 - Sandesh Brand 2Sandesh Brand 2Broa...  2020-02-01 -0.716828
47  Segment 1 - Sandesh Brand 2Sandesh Brand 2Broa...  2020-03-01 -0.715470

In [308]:
result.to_csv('result.csv')